# Import Liberaies / and scrape Data

In [60]:
import soccerdata as sd
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque # new
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
 

from sklearn.feature_selection import SelectFromModel

In [61]:
# Create a scraper class instance for  for the 2020/21 & 2021/22
mh = sd.MatchHistory(leagues=['GER-Bundesliga'], seasons=['2020-2021'])
mh2= sd.MatchHistory(leagues=['GER-Bundesliga'], seasons=['2021-2022'])
# Get match data of each team
match_stats= mh.read_games()
test_data = mh2.read_games()

[11/21/24 09:52:16] INFO     Saving cached data to /Users/kevinwildprett/soccerdata/data/MatchHistory ]8;id=351662;file:///Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=503062;file:///Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

                    INFO     Saving cached data to /Users/kevinwildprett/soccerdata/data/MatchHistory ]8;id=319330;file:///Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=558123;file:///Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/soccerdata/_common.py#89\89]8;;\

In [62]:
# Convert MultiIndex into regular one
match_stats = match_stats.reset_index()
match_stats.reset_index(drop=True, inplace=True)

test_data = test_data.reset_index()
test_data.reset_index(drop=True, inplace=True)


new dataframe for prev season

In [63]:
team = ['Bayern Munich', 'Dortmund', 'RB Leipzig', 'Leverkusen', "M\'gladbach", 'Ein Frankfurt', 'Wolfsburg', 'Hertha', 'Hoffenheim', 'Stuttgart', 'Mainz', 'Freiburg', 'Schalke 04', 'FC Koln', 'Werder Bremen', 'Augsburg', 'Union Berlin', 'Bielefeld']

total_values = ['858.23', '628.40', '574.95', '373.25', '300.75', '269.15', '256.83', '228.88', '228.15', '189.25', '163.30', '139.10', '123.05', '118.20', '111.98', '93.05', '82.05', '56.73']


# create dataframe
df_teams = pd.DataFrame({
    'team': team,
    'total_values': total_values
})

In [64]:
# Merge for home_market_value
match_stats = pd.merge(match_stats,  df_teams[['team', 'total_values']], left_on='home_team', right_on='team').rename(columns={'total_values': 'home_value'})
match_stats = match_stats.drop(columns=['team'])

In [65]:
# Merge for away_market_value
match_stats = pd.merge(match_stats,  df_teams[['team', 'total_values']], left_on='away_team', right_on='team').rename(columns={'total_values': 'away_value'})
match_stats = match_stats.drop(columns=['team'])

new dataframe for test dataframe

In [66]:
team = ['Bayern Munich', 'Dortmund', 'RB Leipzig', 'Leverkusen', 'Wolfsburg', "M\'gladbach", 'Ein Frankfurt', 'Hoffenheim', 'Freiburg', 'Stuttgart', 'Hertha', 'Mainz', 'Union Berlin', 'FC Koln', 'Augsburg', 'Bielefeld', 'Bochum', 'Greuther Furth']

total_values = ['790.33', '558.98', '499.95', '455.05', '269.20', '236.80', '228.15', '211.43', '176.35', '174.23', '155.78', '131.03', '115.90', '99.28', '95.30', '65.85', '52.25', '36.95']

# create dataframe
df_teams2 = pd.DataFrame({
    'team': team,
    'total_values': total_values
})

In [67]:
# Merge for home_market_value
test_data = pd.merge(test_data,  df_teams2[['team', 'total_values']], left_on='home_team', right_on='team').rename(columns={'total_values': 'home_value'})
test_data = test_data.drop(columns=['team'])

In [68]:
# Merge for away_market_value
test_data = pd.merge(test_data,  df_teams2[['team', 'total_values']], left_on='away_team', right_on='team').rename(columns={'total_values': 'away_value'})
test_data = test_data.drop(columns=['team'])

In [69]:
match_stats = match_stats.sort_values(by='date').reset_index(drop=True)
test_data = test_data.sort_values(by='date').reset_index(drop=True)

In [70]:
match_stats.columns.values

array(['league', 'season', 'game', 'date', 'home_team', 'away_team',
       'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST',
       'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH',
       'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH',
       'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5',
       'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5',
       'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA',
       'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD',
       'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH',
       'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA',
       'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5',
       'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh',
       'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA',

# Bulding new Features

Matchdays

In [71]:
# Function to calculate matchdays for each team
def update_matchday(row,team_matchday_counts):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Increment matchday count for home and away teams
    team_matchday_counts[home_team] += 1
    team_matchday_counts[away_team] += 1
    
    # Assign matchday counts to the row
    row['home_team_matchday'] = team_matchday_counts[home_team]
    row['away_team_matchday'] = team_matchday_counts[away_team]
    
    return row

In [72]:
# Apply the function to the first DataFrame with a separate dictionary
team_matchday_counts_df1 = defaultdict(int)
match_stats = match_stats.apply(lambda row: update_matchday(row, team_matchday_counts_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_matchday_counts_df2 = defaultdict(int)
test_data = test_data.apply(lambda row: update_matchday(row, team_matchday_counts_df2), axis=1)

Cumulative Points (positiv effect on pred)

In [73]:
# Function to update cumulative points for each team
def update_cumulative_points(row, team_points):
    home_team = row['home_team']
    away_team = row['away_team']
    home_matchday = row['home_team_matchday']
    away_matchday = row['away_team_matchday']
    
    # Initialize cumulative points as 0 for the first matchday
    if home_matchday == 1:
        row['home_cumulative_points'] = 0
    else:
        row['home_cumulative_points'] = team_points[home_team]
    
    if away_matchday == 1:
        row['away_cumulative_points'] = 0
    else:
        row['away_cumulative_points'] = team_points[away_team]
    
    # Update points based on match result (add to history after setting current cumulative points)
    if row['FTR'] == 'H':  # Home team wins
        team_points[home_team] += 3  # 3 points for home team
        team_points[away_team] += 0  # 0 points for away team
    elif row['FTR'] == 'A':  # Away team wins
        team_points[home_team] += 0  # 0 points for home team
        team_points[away_team] += 3  # 3 points for away team
    else:  # Draw
        team_points[home_team] += 1  # 1 point for home team
        team_points[away_team] += 1  # 1 point for away team
    
    return row

In [74]:
# Apply the function to the first DataFrame with a separate dictionary
team_points_df1 = defaultdict(int)
match_stats = match_stats.apply(lambda row: update_cumulative_points(row, team_points_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_points_df2 = defaultdict(int)
test_data = test_data.apply(lambda row: update_cumulative_points(row, team_points_df2), axis=1)

Wins in last 5 matchdays (negative effect on pred)

In [75]:
# Function to update the last 5 match history and count wins
def update_last_5_wins(row, team_last_5_results):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the number of wins in the last 5 matches or fewer if not enough matches have been played
    row['home_last_5_wins'] = sum(team_last_5_results[home_team])
    row['away_last_5_wins'] = sum(team_last_5_results[away_team])
    
    # Add current match result to history (after calculation)
    if row['FTR'] == 'H':  # Home team wins
        team_last_5_results[home_team].append(1)
        team_last_5_results[away_team].append(0)
    elif row['FTR'] == 'A':  # Away team wins
        team_last_5_results[home_team].append(0)
        team_last_5_results[away_team].append(1)
    else:  # Draw
        team_last_5_results[home_team].append(0)
        team_last_5_results[away_team].append(0)
    
    return row



In [76]:
# Apply the function to the first DataFrame with a separate dictionary
team_last_5_results_df1 = defaultdict(lambda: deque(maxlen=5))
match_stats = match_stats.apply(lambda row: update_last_5_wins(row, team_last_5_results_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_last_5_results_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: update_last_5_wins(row, team_last_5_results_df2), axis=1)

Shots on Goal (negative effect on pred)

In [77]:
# Function to update the number of shots on target for each team in the last 5 matches
def update_last_5_shots(row, team_last_5_shots):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the total shots on target in the last 5 matches or fewer if not enough matches have been played
    row['home_last_5_shots'] = sum(team_last_5_shots[home_team])
    row['away_last_5_shots'] = sum(team_last_5_shots[away_team])
    
    # Add current match shots on target to history
    team_last_5_shots[home_team].append(row['HST'])  # Home shots on target
    team_last_5_shots[away_team].append(row['AST'])  # Away shots on target
    
    return row

In [78]:
# Apply the function to the first DataFrame with a separate dictionary
team_last_5_shots_df1 = defaultdict(lambda: deque(maxlen=5))
match_stats = match_stats.apply(lambda row: update_last_5_shots(row, team_last_5_shots_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_last_5_shots_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: update_last_5_shots(row, team_last_5_shots_df2), axis=1)

Goaldifference (possitive effect on pred)

In [79]:
# Function to update the goal difference for each team in the last 5 matches
def update_last_5_goal_diff(row, team_last_5_goal_diff):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the current match goal difference
    home_goal_diff = row['FTHG'] - row['FTAG']
    away_goal_diff = -home_goal_diff  # Invert for the away team

    # Calculate the total goal difference in the last 5 matches or fewer if not enough matches have been played
    row['home_last_5_goal_diff'] = sum(team_last_5_goal_diff[home_team])
    row['away_last_5_goal_diff'] = sum(team_last_5_goal_diff[away_team])
    
    # Add current match goal difference to history
    team_last_5_goal_diff[home_team].append(home_goal_diff)
    team_last_5_goal_diff[away_team].append(away_goal_diff)
    
    return row

In [80]:
# Apply the function to the first DataFrame with a separate dictionary
team_last_5_goal_diff_df1 = defaultdict(lambda: deque(maxlen=5))
match_stats = match_stats.apply(lambda row: update_last_5_goal_diff(row, team_last_5_goal_diff_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_last_5_goal_diff_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: update_last_5_goal_diff(row, team_last_5_goal_diff_df2), axis=1)

Amount halftime goals (possitive effect on pred)

In [81]:
# Function to calculate and update the sum of half-time goals for the last 5 matches for each team
def calculate_last_5_half_time_goals(row, team_last_5_half_time_goals):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the sum of half-time goals in the last 5 matches or fewer if not enough matches have been played
    row['home_last_5_half_time_goals'] = sum(team_last_5_half_time_goals[home_team])
    row['away_last_5_half_time_goals'] = sum(team_last_5_half_time_goals[away_team])
    
    # Add the current match half-time goals to history
    team_last_5_half_time_goals[home_team].append(row['HTHG'])  # Home half-time goals
    team_last_5_half_time_goals[away_team].append(row['HTAG'])  # Away half-time goals
    
    return row

In [82]:
# Apply the function to the first DataFrame with a separate dictionary
team_last_5_half_time_goals_df1 = defaultdict(lambda: deque(maxlen=5))
match_stats = match_stats.apply(lambda row: calculate_last_5_half_time_goals(row, team_last_5_half_time_goals_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_last_5_half_time_goals_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: calculate_last_5_half_time_goals(row, team_last_5_half_time_goals_df2), axis=1)


Corner shots (possitive effect on pred)

In [83]:
# Function to calculate and update the sum of shot corners for the last 5 matches for each team
def calculate_last_5_corners_shot(row, team_last_5_corners_shot):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the sum of Corners in the last 5 matches or fewer if not enough matches have been played
    row['home_last_5_corners_shot'] = sum(team_last_5_corners_shot[home_team])
    row['away_last_5_corners_shot'] = sum(team_last_5_corners_shot[away_team])
    
    # Add the current match corners to history
    team_last_5_corners_shot[home_team].append(row['HC'])  # Home Corners
    team_last_5_corners_shot[away_team].append(row['AC'])  # Away Corners
    
    return row


In [84]:
# Apply the function to the first DataFrame with a separate dictionary
team_last_5_corners_shot_df1 = defaultdict(lambda: deque(maxlen=5))
match_stats = match_stats.apply(lambda row: calculate_last_5_corners_shot(row, team_last_5_corners_shot_df1), axis=1)

# Apply the function to the second DataFrame with a new dictionary
team_last_5_corners_shot_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: calculate_last_5_corners_shot(row, team_last_5_corners_shot_df2), axis=1)

ELO-Ratings (possitive effect on pred)

In [85]:
### for train data ###
# Dictionary to keep track of current Elo ratings for each team
# Reset Elo ratings and ensure initial value is 1500 for all teams
# Re-initialize Elo calculation to ensure correct data
team_elo = defaultdict(lambda: 1500)
K=32

# Lists to store Elo ratings for current matchdays
current_home_elo_list = []
current_away_elo_list = []

# Iterate over each match to calculate Elo ratings correctly
for index, row in match_stats.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']

    # Get the current Elo ratings from the dictionary
    current_home_elo = team_elo[home_team]
    current_away_elo = team_elo[away_team]

    # Append the current Elo ratings to the lists
    current_home_elo_list.append(current_home_elo)
    current_away_elo_list.append(current_away_elo)

    # Calculate expected outcomes
    expected_home = 1 / (1 + 10 ** ((current_away_elo - current_home_elo) / 400))
    expected_away = 1 / (1 + 10 ** ((current_home_elo - current_away_elo) / 400))

    # Determine actual match outcome
    if row['FTHG'] > row['FTAG']:  # Home win
        actual_home = 1
        actual_away = 0
    elif row['FTHG'] < row['FTAG']:  # Away win
        actual_home = 0
        actual_away = 1
    else:  # Draw
        actual_home = 0.5
        actual_away = 0.5

    # Update Elo ratings using the K factor
    new_home_elo = current_home_elo + K * (actual_home - expected_home)
    new_away_elo = current_away_elo + K * (actual_away - expected_away)

    # Update the dictionary with the new Elo ratings for the next match
    team_elo[home_team] = new_home_elo
    team_elo[away_team] = new_away_elo

# Add the current Elo ratings to the DataFrame
match_stats['current_home_elo'] = current_home_elo_list
match_stats['current_away_elo'] = current_away_elo_list

/var/folders/n_/wh82d8z12pjcn_v99zhyg1t80000gn/T/ipykernel_51557/4152494630.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_stats['current_home_elo'] = current_home_elo_list
/var/folders/n_/wh82d8z12pjcn_v99zhyg1t80000gn/T/ipykernel_51557/4152494630.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_stats['current_away_elo'] = current_away_elo_list


In [86]:
### 4 test data ###
# Dictionary to keep track of current Elo ratings for each team
# Reset Elo ratings and ensure initial value is 1500 for all teams
# Re-initialize Elo calculation to ensure correct data
team_elo = defaultdict(lambda: 1500)
K=32

# Lists to store Elo ratings for current matchdays
current_home_elo_list = []
current_away_elo_list = []

# Iterate over each match to calculate Elo ratings correctly
for index, row in test_data.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']

    # Get the current Elo ratings from the dictionary
    current_home_elo = team_elo[home_team]
    current_away_elo = team_elo[away_team]

    # Append the current Elo ratings to the lists
    current_home_elo_list.append(current_home_elo)
    current_away_elo_list.append(current_away_elo)

    # Calculate expected outcomes
    expected_home = 1 / (1 + 10 ** ((current_away_elo - current_home_elo) / 400))
    expected_away = 1 / (1 + 10 ** ((current_home_elo - current_away_elo) / 400))

    # Determine actual match outcome
    if row['FTHG'] > row['FTAG']:  # Home win
        actual_home = 1
        actual_away = 0
    elif row['FTHG'] < row['FTAG']:  # Away win
        actual_home = 0
        actual_away = 1
    else:  # Draw
        actual_home = 0.5
        actual_away = 0.5

    # Update Elo ratings using the K factor
    new_home_elo = current_home_elo + K * (actual_home - expected_home)
    new_away_elo = current_away_elo + K * (actual_away - expected_away)

    # Update the dictionary with the new Elo ratings for the next match
    team_elo[home_team] = new_home_elo
    team_elo[away_team] = new_away_elo

# Add the current Elo ratings to the DataFrame
test_data['current_home_elo'] = current_home_elo_list
test_data['current_away_elo'] = current_away_elo_list

/var/folders/n_/wh82d8z12pjcn_v99zhyg1t80000gn/T/ipykernel_51557/1577465140.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['current_home_elo'] = current_home_elo_list
/var/folders/n_/wh82d8z12pjcn_v99zhyg1t80000gn/T/ipykernel_51557/1577465140.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['current_away_elo'] = current_away_elo_list


streak (possitive effect on pred)

In [87]:
# Function to calculate and update the winning streak for each team
def calculate_streak(row, team_streaks):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Store the current winning streak before updating it, so it appears for the next matchday
    row['home_team_streak'] = team_streaks[home_team]
    row['away_team_streak'] = team_streaks[away_team]
    
    # Update the winning streak based on the result of the current matchday
    if row['FTR'] == 'H':  # Home team wins
        team_streaks[home_team] += 1  # Increment home team winning streak
        team_streaks[away_team] = 0   # Reset away team streak
    elif row['FTR'] == 'A':  # Away team wins
        team_streaks[away_team] += 1  # Increment away team winning streak
        team_streaks[home_team] = 0   # Reset home team streak
    else:  # Draw
        team_streaks[home_team] = 0   # Reset both streaks
        team_streaks[away_team] = 0

    return row

In [88]:
# Create a dictionary to track the winning streaks for each team
team_streaks_df1 = defaultdict(int)

# Wende die Funktion auf den ersten DataFrame an
match_stats = match_stats.apply(lambda row: calculate_streak(row, team_streaks_df1), axis=1)

# Für den zweiten DataFrame ein neues Dictionary verwenden
team_streaks_df2 = defaultdict(int)
test_data = test_data.apply(lambda row: calculate_streak(row, team_streaks_df2), axis=1)

goals last 5 matches (negative effect on streak)

In [89]:
# Function to calculate the total goals scored in the last 5 matches
def calculate_last_5_goals(row, team_goals_history):
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Calculate the total goals for the last 5 matches
    row['home_last_5_goals'] = sum(team_goals_history[home_team])
    row['away_last_5_goals'] = sum(team_goals_history[away_team])
    
    # Update the goals history for the current match
    team_goals_history[home_team].append(row['FTHG'])  # Full Time Home Goals
    team_goals_history[away_team].append(row['FTAG'])  # Full Time Away Goals
    
    return row


In [90]:
# Create a dictionary to track the last 5 goals for each team for the first DataFrame
team_goals_history_df1 = defaultdict(lambda: deque(maxlen=5))

# Apply the function to the first DataFrame
match_stats = match_stats.apply(lambda row: calculate_last_5_goals(row, team_goals_history_df1), axis=1)

# For the second DataFrame, use a new dictionary
team_goals_history_df2 = defaultdict(lambda: deque(maxlen=5))
test_data = test_data.apply(lambda row: calculate_last_5_goals(row, team_goals_history_df2), axis=1)

In [91]:
# Creats a Excel Sheet as a output
new= match_stats[['home_team','away_team','FTHG','FTAG','HTHG','HTAG','HST' ,'AST','HC', 'AC', 'home_team_matchday', 'away_team_matchday', 'home_last_5_goals', 'away_last_5_goals' ]]
new.to_excel('stats_mit_Features.xlsx', index=False)

new= test_data[['home_team','away_team','FTHG','FTAG','HTHG','HTAG','HST' ,'AST','HC', 'AC','home_team_matchday', 'away_team_matchday', 'home_last_5_goals', 'away_last_5_goals']]
new.to_excel('stats_mit_Features_test.xlsx', index=False)

 # Feature Engineering

In [92]:
 prediction_df = match_stats[['current_home_elo', 'current_away_elo', 'FTR',
                            'home_last_5_corners_shot', 'away_last_5_corners_shot', 
                             'home_value', 'away_value','home_last_5_goal_diff', 'away_last_5_goal_diff',
                             'home_cumulative_points', 'away_cumulative_points',
                            'home_last_5_half_time_goals', 'away_last_5_half_time_goals',
                            'home_team_matchday', 'away_team_matchday', 'home_team_streak', 'away_team_streak'
                             ]]


In [93]:
test_df = test_data[[ 'current_home_elo', 'current_away_elo', 'FTR', 
                     'home_last_5_corners_shot', 'away_last_5_corners_shot', 
                     'home_value', 'away_value', 'home_last_5_goal_diff', 'away_last_5_goal_diff',   
                     'home_cumulative_points', 'away_cumulative_points', 
                    'home_last_5_half_time_goals', 'away_last_5_half_time_goals',
                    'home_team_matchday', 'away_team_matchday', 'home_team_streak', 'away_team_streak'
                    ]]

In [94]:
le = LabelEncoder()

#Test/ Train split
y_train = le.fit_transform(prediction_df['FTR'])
X_train = prediction_df.drop(columns=['FTR'])   
y_test = le.fit_transform(test_df['FTR'])
X_test = test_df.drop(columns=['FTR'])  

label_mapping = dict(zip(le.transform(le.classes_), le.classes_))
print("Label mapping:", label_mapping)




Label mapping: {0: 'A', 1: 'D', 2: 'H'}


## Implementing Random Forest + Training

In [95]:
# Initialize the model, right now without optimal hyperparameter
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, class_weight='balanced')
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

gameday = 1
accuracies = []  # Store accuracy for each matchday

# Iterative training process
while not X_test.empty:
    
    # Extract the first matchday (9 games)
    X_test_batch = X_test.iloc[:9]
    y_test_batch = y_test[:9]

    # Train the model and calculate accuracy
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test_batch)
    accuracy = accuracy_score(y_test_batch, y_pred)
    accuracies.append(accuracy)
    print(f"Gameday {gameday}: Accuracy on Testdata = {accuracy:.2f}")

    # Display predictions and actual values when accuracy is low
    if accuracy < 0.5:
        print("\nDetailed Results for Gameday", gameday)
        results_df = X_test_batch.copy()
        results_df['Predicted'] = y_pred
        results_df['Actual'] = y_test_batch
        print(results_df[['Predicted', 'Actual']])

    # Add the 9 test data points to the training dataset
    X_train = np.vstack([X_train, X_test_batch])
    y_train = np.concatenate([y_train, y_test_batch])

    # Remove the used test data points from X_test
    X_test = X_test.iloc[9:]
    y_test = y_test[9:]

    gameday += 1

# Extract feature importances
feature_importances = rf_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Display the importance
print(importance_df)

# Calculate the overall accuracy
overall_accuracy = np.mean(accuracies)
print(f"\nOverall Accuracy: {overall_accuracy:.5f}")

Gameday 1: Accuracy on Testdata = 0.22

Detailed Results for Gameday 1
   Predicted  Actual
0          0       1
1          0       1
2          1       2
3          0       1
4          0       2
5          0       0
6          2       2
7          0       2
8          0       2


/Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Gameday 2: Accuracy on Testdata = 0.56


/Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Gameday 3: Accuracy on Testdata = 0.56


...

/Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Gameday 32: Accuracy on Testdata = 0.33

Detailed Results for Gameday 32
     Predicted  Actual
279          2       1
280          1       1
281          0       0
282          0       2
283          2       0
284          2       1
285          2       0
286          0       2
287          2       2


/Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Gameday 33: Accuracy on Testdata = 0.56
Gameday 34: Accuracy on Testdata = 0.44

Detailed Results for Gameday 34
     Predicted  Actual
297          0       1
298          2       1
299          2       2
300          2       2
301          2       2
302          2       2
303          0       2
304          2       1
305          0       2
                        Feature  Importance
1              current_away_elo    0.097699
4                    home_value    0.095256
5                    away_value    0.089719
0              current_home_elo    0.088649
2      home_last_5_corners_shot    0.085336
6         home_last_5_goal_diff    0.068609
3      away_last_5_corners_shot    0.067752
7         away_last_5_goal_diff    0.065710
9        away_cumulative_points    0.057741
8        home_cumulative_points    0.057046
11  away_last_5_half_time_goals    0.045475
10  home_last_5_half_time_goals    0.045286
13           away_team_matchday    0.043628
12           home_team_matchday    0.0405

/Users/kevinwildprett/Desktop/schule/miniconda3/envs/ml-course/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


## Hyperparameter search ##

make cell under as a code cell und the upper one to a markdown cell to run

# Define the parameter grid for hyperparameter search
param_dist_extended = {
    'n_estimators': [100, 500, 1000, 1500, 2000],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'class_weight': ['balanced', 'balanced_subsample', None]
}
# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100,random_state=42,max_depth=10, class_weight='balanced' )

# Execute RandomizedSearchCV once
random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=param_dist_extended,
    n_iter=50,               # Number of combinations to test
    cv=5,                    # 5-fold cross-validation
    scoring='accuracy',      # Scoring metric
    n_jobs=-1,               # Use all available processors
    random_state=42, 
    verbose=2
)

# Apply hyperparameter search on training data
random_search.fit(X_train, y_train)

# Store the best parameters in a variable
best_params = random_search.best_params_

# Optionally, output the best parameters and cross-validation score
print("Best Hyperparameters:", best_params)
print("Best Cross-Validation Score:", random_search.best_score_)

# Define feature names based on X_train's type
if isinstance(X_train, pd.DataFrame):
    feature_names = X_train.columns
else:
    feature_names = [f'Feature_{i}' for i in range(X_train.shape[1])]

# Use the best parameters to create a new model
best_rf_model = RandomForestClassifier(**best_params, random_state=42)

# Use the best model instance for iterative training
best_rf_model.fit(X_train, y_train)
y_pred = best_rf_model.predict(X_test)
gameday = 1
accuracies = []  # List to store accuracy for each game day

# Iterative process
while len(X_test) > 0:
    # Extract the first matchday (9 games)
    X_test_batch = X_test[:9]
    y_test_batch = y_test[:9]

    # Train the model and calculate accuracy for the current matchday
    best_rf_model.fit(X_train, y_train)
    y_pred = best_rf_model.predict(X_test_batch)
    accuracy = accuracy_score(y_test_batch, y_pred)
    accuracies.append(accuracy)  # Store accuracy of the current matchday
    print(f"Gameday {gameday}: Accuracy on Testdata = {accuracy:.2f}")

    # If accuracy is low or gameday equals 4, show predictions and actual values
    if accuracy < 0.5:
        print("\nDetailed Results for Gameday", gameday)
        results_df = pd.DataFrame(X_test_batch, columns=feature_names)
        results_df['Predicted'] = y_pred
        results_df['Actual'] = y_test_batch
        print(results_df[['Predicted', 'Actual']])  # Show predictions and actual values

    # Add the 9 test data points to the training set
    X_train = np.vstack([X_train, X_test_batch]) 
    y_train = np.concatenate([y_train, y_test_batch])  

    # Remove the used test data points from X_test
    X_test = X_test[9:]
    y_test = y_test[9:]
    
    gameday += 1

# Calculate the overall accuracy
overall_accuracy = np.mean(accuracies)
print(f"\nOverall Accuracy across all gamedays: {overall_accuracy:.6f}")